In [1]:
import gurobipy

In [2]:
gurobipy.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-05


<gurobi.Model Continuous instance Unnamed: 0 constrs, 0 vars, Parameter changes: Username=(user-defined)>